In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sts
from scipy import constants 
%matplotlib inline

In [20]:
# Parametersfor the benchmarking
fps1=100.0
num_particles=1000                       
dt=1.0/fps1
spatial_resolution=1.0E-8  # in metres per px
dust_diameter=7.14E-6 # in metres
dust_rho=1510.0
mu = 0
mass = 4.0 / 3.0 * np.pi * ( dust_diameter/ 2.0)**3 * dust_rho #mass of the dust particles
# print(mass)
kb=constants.Boltzmann
T=1200 #temperature in Kelvin
sigma = np.sqrt(kb*T/mass)
print(sigma)

left=0
right=1751 # right border in px as per the camera of expt
up=400 # up border in px as per the camera of expt
down=0
left_SI=left*spatial_resolution
right_SI=right*spatial_resolution
up_SI=up*spatial_resolution
down_SI=down*spatial_resolution
final_frame=2
n_bins=19

0.0002399369654438579


In [14]:
def make_velocities(fps,num_particles,spatial_resolution, mu,T, right_border_px, up_border_px,final_frame):
    dt=1.0/fps
    init_frame=1
    dust_diameter=7.14E-6 
    dust_rho=1510.0
    mass = 4.0 / 3.0 * np.pi * (dust_diameter/ 2.0)**3 * dust_rho #mass of the dust particles
    # print(mass)
    kb=constants.Boltzmann
    sigma = np.sqrt(kb*T/mass)
    # print(sigma)
    left=0
    down=0
    down_SI=down*spatial_resolution
    left_SI=left*spatial_resolution
    up_SI=up_border_px*spatial_resolution
    right_SI=right_border_px*spatial_resolution
    frame_full_df=pd.DataFrame()
    while init_frame<=final_frame:
        if init_frame==1:
            # Generate a sample of 1000 random numbers from a uniform distribution to act as the x coordinates of the particles
            unif_x = sts.uniform(left_SI, right_SI-left_SI)
            sample_x=unif_x.rvs(num_particles)
            sample_x_in_resol= sample_x/spatial_resolution
            sample_x_trunc= sample_x_in_resol.astype('int64')
            sample_x_trunc_SI=sample_x_trunc.astype('float64')*spatial_resolution
            
            unif_y = sts.uniform(down_SI, up_SI-down_SI)
            sample_y=unif_y.rvs(num_particles)
            sample_y_in_resol= sample_y/spatial_resolution
            sample_y_trunc= sample_y_in_resol.astype('int64')
            sample_y_trunc_SI=sample_y_trunc.astype('float64')*spatial_resolution

            part_no = np.arange(0,num_particles,dtype = 'int') # array of particles ID numbers
            # Generate a sample of 1000 random velocities from a normal distribution
            norm_rv = sts.norm(mu, sigma)
            sample_vx = norm_rv.rvs(num_particles) #generate a sample of certain size
            sample_vy = norm_rv.rvs(num_particles) #generate a sample of certain size
            
        # Simulating the motion of the particles for x coordinates
        x_forward = sample_x_trunc_SI + sample_vx*dt
        x_forward_in_resol = x_forward / spatial_resolution
        x_forward_trunc = x_forward_in_resol.astype('int64')
        x_forward_SI = x_forward_trunc.astype('float64') * spatial_resolution
        # Simulating the motion of the particles for y coordinates
        y_forward = sample_y_trunc_SI + sample_vy * dt
        y_forward_in_resol = y_forward / spatial_resolution
        y_forward_trunc = y_forward_in_resol.astype('int64')
        y_forward_SI=y_forward_trunc.astype('float64')*spatial_resolution

        # Calculating restored velocities
        v_restored_x=(x_forward_SI- sample_x_trunc_SI)/dt
        v_restored_y=(y_forward_SI- sample_y_trunc_SI)/dt
        # Append the data for the current frame
        frame_data={'part_no':part_no, 'x': sample_x_trunc_SI, 'y':sample_y_trunc_SI, 'frame': init_frame,'vx (restored)': v_restored_x, 'vy (restored)': v_restored_y, 'vx': sample_vx, 'vy': sample_vy}
        frame_df=pd.DataFrame(frame_data)
        frame_full_df = pd.concat([frame_full_df, frame_df], ignore_index=True)
        sample_x_trunc_SI=x_forward_SI
        sample_y_trunc_SI= y_forward_SI
        init_frame+=1
    return frame_full_df

In [21]:
x_y_df = make_velocities(fps1,num_particles,spatial_resolution, mu,T, right, up, 2)
x_y_test = x_y_df
x_y_test= x_y_test[['part_no', 'x', 'y','frame']]
x_y_test = x_y_test.rename(columns={'part_no': 'particle'})
x_y_test

,particle,x,y,frame
0,0,0.000003,1.690000e-06,1
1,1,0.000003,5.400000e-07,1
2,2,0.000013,3.770000e-06,1
3,3,0.000007,3.480000e-06,1
4,4,0.000016,1.830000e-06,1
...,...,...,...,...
1995,995,0.000002,2.420000e-06,2
1996,996,0.000004,4.350000e-06,2
1997,997,0.000015,1.320000e-06,2
1998,998,0.000001,3.920000e-06,2


In [16]:
#function get_velocities(df, step) calculates velocites of particles with a given step step.
def get_velocities(df, step):
#initialize empy arrays to store data:
    arr_particle = np.array([])
    arr_x = np.array([])
    arr_y = np.array([])
    arr_vx = np.array([])
    arr_vy = np.array([])
    arr_frame = np.array([])
    # get an array containing all frame numbers in the input dataframe:
    frames_listing = np.unique(np.array(df['frame']))
    #cycle throught all those frames:
    for iFrame in range(step, len(frames_listing)):
        #get current frame:
        cur_frame = frames_listing[iFrame]
        #select a dataframe containing data ONLY for that frame:
        df_front_frame = df[(df['frame'] == cur_frame)]
        print(iFrame)
        #cycle throught all particles in the frame and find their velocities as position of the particle in that frame minus position of the same particles step frames ago:
        for i in range(0, len(df_front_frame)):
            #take i-th particle in a frame
            cur_p = df_front_frame['particle'].iloc[i]
            cur_x = df_front_frame['x'].iloc[i]
            cur_y = df_front_frame['y'].iloc[i]
            #find a row with the same particle in a frame step frames ago:
            prev_frame_cur_row = df[((df['frame'] == cur_frame - step) & (df['particle'] == cur_p))]
            #if that particle excisted back then, we will get exactly ONE row:
            if (len(prev_frame_cur_row) == 1):
                #if this row exists, we can take position of that particle in that, previous, frame:
                prev_x = prev_frame_cur_row['x'].iloc[0]
                prev_y = prev_frame_cur_row['y'].iloc[0]
                # so we can calculate velocities:
                cur_vx = cur_x - prev_x
                cur_vy = cur_y - prev_y
                cur_particle = df_front_frame['particle'].iloc[i]
                #and append all parameters of that particle to our data arrays
                arr_vx = np.append(arr_vx, cur_vx)
                arr_vy = np.append(arr_vy, cur_vy)
                arr_particle = np.append(arr_particle, cur_particle)
                arr_x = np.append(arr_x, cur_x)
                arr_y = np.append(arr_y, cur_y)
                arr_frame = np.append(arr_frame, cur_frame)
    #save output as a dataframe containing all the info we need:
    data = {'frame':arr_frame, 'particle':arr_particle, 'x': arr_x, 'y': arr_y, 'vx': arr_vx, 'vy':arr_vy}
    ret_df = pd.DataFrame(data)
                
    return ret_df

In [22]:
df_velocities = get_velocities(x_y_test, 1)
df_velocities

1


,frame,particle,x,y,vx,vy
0,2.0,0.0,0.000003,8.400000e-07,3.000000e-07,-8.500000e-07
1,2.0,1.0,0.000004,2.540000e-06,9.300000e-07,2.000000e-06
2,2.0,2.0,0.000009,8.490000e-06,-4.180000e-06,4.720000e-06
3,2.0,3.0,0.000006,6.630000e-06,-1.020000e-06,3.150000e-06
4,2.0,4.0,0.000013,1.340000e-06,-3.130000e-06,-4.900000e-07
...,...,...,...,...,...,...
995,2.0,995.0,0.000002,2.420000e-06,-8.700000e-07,8.400000e-07
996,2.0,996.0,0.000004,4.350000e-06,1.500000e-07,1.010000e-06
997,2.0,997.0,0.000015,1.320000e-06,-2.240000e-06,-1.430000e-06
998,2.0,998.0,0.000001,3.920000e-06,-2.430000e-06,2.280000e-06


In [23]:
std1= np.std(df_velocities['vx'])
std_SI=std1/dt
print(std_SI)
T_def_vel=mass*std_SI**2/constants.Boltzmann
print(T_def_vel)
print(T_def_vel/11604.0)  # convert to eV


0.00023690277795754118
1169.8420525065856
0.10081368946109838


In [24]:
T_make_vel=mass*np.std(x_y_df['vx (restored)'])**2/constants.Boltzmann
print(T_make_vel)
print(T_def_vel)
print(np.abs(T_make_vel-T)/T)
print(np.abs(T_def_vel-T)/T)


1169.527860716259
1169.8420525065856
0.025393449403117455
0.0251316229111787
